In [0]:
# imports

import os
import re
import math
import json
from tqdm import tqdm
import random
from dotenv import load_dotenv
from huggingface_hub import login
import matplotlib.pyplot as plt
import numpy as np
import pickle
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import chromadb
from items import Item
from testing import Tester

In [0]:
# environment
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [0]:
openai = OpenAI()

In [0]:
# Load in the test pickle file
with open('test.pkl', 'rb') as file:
    test = pickle.load(file)

In [0]:
def make_context(similars, prices):
    message = "To provide some context, here are some other items that might be similar to the item you need to estimate.\n\n"
    for similar, price in zip(similars, prices):
        message += f"Potentially related product:\n{similar}\nPrice is ${price:.2f}\n\n"
    return message

In [0]:
def messages_for(item, similars, prices):
    system_message = "You estimate prices of items. Reply only with the price, no explanation. Price is always below $1000."
    user_prompt = make_context(similars, prices)
    user_prompt += "And now the question for you:\n\n"
    user_prompt += item.test_prompt().replace(" to the nearest dollar","").replace("\n\nPrice is $","")
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": "Price is $"}
    ]

In [0]:
DB = "products_vectorstore"

In [0]:
client = chromadb.PersistentClient(path=DB)
collection = client.get_or_create_collection('products')

In [0]:
def description(item):
    text = item.prompt.replace("How much does this cost to the nearest dollar?\n\n", "")
    return text.split("\n\nPrice is $")[0]

In [0]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [0]:
def vector(item):
    return model.encode([description(item)])

In [0]:
def find_similars(item):
    results = collection.query(query_embeddings=vector(item).astype(float).tolist(), n_results=5)
    documents = results['documents'][0][:]
    prices = [m['price'] for m in results['metadatas'][0][:]]
    return documents, prices

In [0]:
print(test[1].prompt)

In [0]:
documents, prices = find_similars(test[1])

In [0]:
print(make_context(documents, prices))

In [0]:
print(messages_for(test[1], documents, prices))

In [0]:
def get_price(s):
    s = s.replace('$','').replace(',','')
    match = re.search(r"[-+]?\d*\.\d+|\d+", s)
    return float(match.group()) if match else 0

In [0]:
get_price("The price for this is $99.99")

In [0]:
# The function for gpt-4o-mini

def gpt_4o_mini_rag(item):
    documents, prices = find_similars(item)
    response = openai.chat.completions.create(
        model="gpt-4o-mini", 
        messages=messages_for(item, documents, prices),
        seed=42,
        max_tokens=5
    )
    reply = response.choices[0].message.content
    return get_price(reply)

In [0]:
# The function for gpt-4.1

# def gpt_4_1_rag(item):
#     documents, prices = find_similars(item)
#     response = openai.chat.completions.create(
#         model="gpt-4.1", 
#         messages=messages_for(item, documents, prices),
#         seed=42,
#         max_tokens=5
#     )
#     reply = response.choices[0].message.content
#     return get_price(reply)

In [0]:
gpt_4o_mini_rag(test[1])

In [0]:
# gpt_4_1_rag(test[1])

In [0]:
test[1].price

In [0]:
Tester.test(gpt_4o_mini_rag, test)